# Michael Manual Trading

## Simulation (brute force)
The idea of standard deviation optimization is as following:
- Teams will choose the cargos with the highest profit.
- Cargos decrease in profitability as more people choose them.
- If the expected value of a cargo is higher for one cargo, people will switch to that cargo.
Therefore, profits across the cargos should converge to each other, i.e. the standard deviation of the profits should be minimized. The resulting vector should include the "stable state" distribution of people who choose each cargo.

What needs to be changed? We should include something that enables optionality of the second cargo. We should also think carefully about adverse selection. 

In [18]:
import numpy as np
from scipy.optimize import minimize

y = np.array([10, 17, 20, 31, 37, 50, 73, 80, 89, 90]) * 10000 # multipliers
z = np.array([1, 1, 2, 2, 3, 4, 4, 6, 8, 10]) # inhabitants

# meta-distribution
# 40%: 73x
# 20%: 17x
# 10%: 31x
# 5%: 80x
# 5%: others uniformly
# uncertainty 30%

# simulation: we have a distribution
randomly pick 14000 times according to this distribution, give me an array of profit

# goal: pick the most profitable crate assuming some distribution of other's people choices


[100000.         170000.         100000.         155000.
 123333.33333333 125000.         182500.         133333.33333333
 111250.          90000.        ]


In [35]:
import numpy as np
from scipy.optimize import minimize

y = np.array([10, 17, 20, 31, 37, 50, 73, 80, 89, 90]) * 10000 # multipliers
z = np.array([1, 1, 2, 2, 3, 4, 4, 6, 8, 10]) # inhabitants

# Do some optimization on SD
def brute_force():
    
    def variance_of_p(x, y, z):
        p = y / ((x*100) + z)
        p_mean = np.mean(p)
        return np.mean((p - p_mean)**2)

    def constraint_eq(x):
        return np.sum(x) - 1

    # Bounds: x_i >= 0
    bounds = [(0, 1)] * 10

    # Initial guess (uniform)
    x0 = np.array([0.1 for _ in range(10)])
    
    # Run the optimizer
    result = minimize(
        variance_of_p,
        x0,
        args=(y, z),
        method='Nelder-Mead',
        bounds=bounds,
        constraints={'type': 'eq', 'fun': constraint_eq},
        options={'disp': True}
    )
    
    # Output the optimal x values and corresponding p values
    x_opt = result.x
    p_opt = y / ((x_opt*10) + z)
    std_dev = np.std(p_opt)
    
    print("Optimal x:", x_opt)
    print("Resulting p:", p_opt)
    print("Standard deviation of p:", std_dev)

    return(x_opt)

def profits(test): return np.round(y/((test*10)+z), 3)
brute_force_optimal = np.round(brute_force(), 3)
uniform = np.array([0.1 for _ in range(10)])
weighted_uniform = np.round(np.array([y[_]/z[_]/np.sum(y/z) for _ in range(10)]), 3)

print("\n\n\nMULTIPLIERS FOR REFERENCE: ", y)
print("BRUTE FORCE DISTRIBUTION: ", brute_force_optimal)
print("PROFITABILITY ASSUMING UNIFORM DIST: ", profits(uniform))
print("PROFITABILITY ASSUMING WEIGHTED DIST: ", profits(weighted_uniform))
print("PROFITABILITY ASSUMING OPTIMAL DIST: ", profits(brute_force_optimal))

Optimization terminated successfully.
         Current function value: 0.000072
         Iterations: 1294
         Function evaluations: 1849
Optimal x: [0.0224486  0.04516263 0.04489723 0.08059072 0.09005987 0.12224306
 0.196875   0.19958882 0.20879262 0.19203765]
Resulting p: [ 81666.91702185 117110.03052777  81666.91085308 110481.20275825
  94857.23209081  95740.86139317 122303.66492887 100051.42353681
  88224.2772536   75500.97078218]
Standard deviation of p: 15002.579719427731



MULTIPLIERS FOR REFERENCE:  [100000 170000 200000 310000 370000 500000 730000 800000 890000 900000]
BRUTE FORCE DISTRIBUTION:  [0.022 0.045 0.045 0.081 0.09  0.122 0.197 0.2   0.209 0.192]
PROFITABILITY ASSUMING UNIFORM DIST:  [ 50000.     85000.     66666.667 103333.333  92500.    100000.
 146000.    114285.714  98888.889  81818.182]
PROFITABILITY ASSUMING WEIGHTED DIST:  [ 56497.175  73275.862  72202.166  96875.     93434.343 100603.622
 134935.305 113798.009 100451.467  84112.15 ]
PROFITABILITY ASSUMIN

C:\Users\micha\anaconda3\Lib\site-packages\scipy\optimize\_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


In [50]:
def l_optimize(n):
    x = np.array([0.1 for _ in range(10)])
    y = np.array([10, 17, 20, 31, 37, 50, 73, 80, 89, 90]) * 10000 # multipliers
    z = np.array([1, 1, 2, 2, 3, 4, 4, 6, 8, 10]) # inhabitants

    ctr = 0
    while ctr < n:
        returns = y / (x*100 + z)
        
        max_i = np.argmax(returns)
        min_i = np.argmin(returns)

        if ctr > n - 3:
            print(f"{returns}\n{max_i}\n{min_i}")

        half = (returns[max_i] - returns[min_i]) / 2

        normalize_sum = x[max_i] + x[min_i]

        if ctr > n - 3:
            print(x)
        
        x[max_i] = y[max_i]/half + z[max_i]
        x[min_i] = y[min_i]/half + z[min_i]

        new_sum = x[max_i] + x[min_i]
        
        x[max_i] = x[max_i] * normalize_sum / new_sum
        x[min_i] = x[min_i] * normalize_sum / new_sum

        if ctr > n - 3:
            print(x)
        # print("Sum of weights: ", np.sum(x))

        ctr += 1

    return x
    
np.set_printoptions(suppress=True)

n = 999
print(f"Optimal distribution given {n} iterations: ", np.round(l_optimize(n), 3))
# print(f"Profit distribution given {n} iterations: ", np.round(y/(z+l_optimize(n)*100), 3))

[34687.26107047 35713.55743708 35008.53109974 40712.11273718
 34290.59736047 35714.28571429 35119.46623093 35450.51698671
 35131.56786775 33894.04798383]
3
9
[0.01882903 0.03760097 0.03712893 0.05614441 0.0779013  0.1
 0.16786193 0.16566667 0.17333341 0.16553335]
[0.01882903 0.03760097 0.03712893 0.05614441 0.0779013  0.1
 0.16786193 0.16566667 0.17333341 0.16553335]
[34687.26107047 35713.55743708 35008.53109974 40712.11273718
 34290.59736047 35714.28571429 35119.46623093 35450.51698671
 35131.56786775 33894.04798383]
3
9
[0.01882903 0.03760097 0.03712893 0.05614441 0.0779013  0.1
 0.16786193 0.16566667 0.17333341 0.16553335]
[0.01882903 0.03760097 0.03712893 0.05614441 0.0779013  0.1
 0.16786193 0.16566667 0.17333341 0.16553335]
Optimal distribution given 999 iterations:  [0.019 0.038 0.037 0.056 0.078 0.1   0.168 0.166 0.173 0.166]


In [ ]:
# CHAT GPT SOLUTION

import random
import numpy as np
from collections import defaultdict

def simulate_crate_game(crate_values, num_players, iterations=10000):
    """
    Simulates the crate selection game to find approximate equilibria.
    
    Parameters:
    - crate_values: List of integers representing crate dollar amounts
    - num_players: Integer number of players
    - iterations: Number of simulations to run

    Returns:
    - Best distribution found with player counts per crate
    - Corresponding average payoff per crate
    """
    num_crates = len(crate_values)
    best_distribution = None
    best_min_payoff = 0
    best_avg_payoff = 0

    for _ in range(iterations):
        # Random distribution of players
        distribution = [0] * num_crates
        for _ in range(num_players):
            chosen_crate = random.randint(0, num_crates - 1)
            distribution[chosen_crate] += 1

        # Calculate payoffs
        payoffs = []
        for i in range(num_crates):
            if distribution[i] > 0:
                payoffs.append(crate_values[i] / distribution[i])
            else:
                payoffs.append(0)

        min_payoff = min([p for p in payoffs if p > 0])  # Avoid empty crates
        avg_payoff = sum(payoffs) / num_crates

        # Choose best based on highest minimum payoff, then average
        if min_payoff > best_min_payoff or (min_payoff == best_min_payoff and avg_payoff > best_avg_payoff):
            best_distribution = distribution[:]
            best_min_payoff = min_payoff
            best_avg_payoff = avg_payoff

    # Final payoff for best distribution
    final_payoffs = [crate_values[i] / best_distribution[i] if best_distribution[i] > 0 else 0 for i in range(num_crates)]

    return {
        "crate_values": crate_values,
        "distribution": best_distribution,
        "payoffs": final_payoffs,
        "min_payoff": best_min_payoff,
        "avg_payoff": best_avg_payoff
    }

# Example usage
if __name__ == "__main__":
    crate_values = [10, 20, 30, 40, 50]
    num_players = 10
    result = simulate_crate_game(crate_values, num_players)

    print("Crate values:", result['crate_values'])
    print("Player distribution:", result['distribution'])
    print("Payoffs:", [round(p, 2) for p in result['payoffs']])
    print("Min payoff:", round(result['min_payoff'], 2))
    print("Avg payoff:", round(result['avg_payoff'], 2))
